In [70]:
from ogs6py import ogs
import numpy as np
import ogs6py
import matplotlib.pyplot as plt
import time
import math
import gmsh
import os
import argparse
import re
import time
import csv
import pandas as pd
import glob
import pyvista as pv
from ogstools.msh2vtu import msh2vtu
import shutil
import xml.etree.ElementTree as ET

pi = math.pi
plt.rcParams["text.usetex"] = True

# Swelling


## Problem Description

![Schematic view of hydraulic fracturing problem and Boundary conditions](./figures/Model_propagating_straight.png#one-half "Schematic view of hydraulic fracturing problem and Boundary conditions.")

# Input Data

The simulations were run with the properties listed in the Table below.


| **Name**                       | **Range**                 | **Unit**   | **Symbol** |
|--------------------------------|---------------------------|------------|------------|
| _Young's modulus_              | 500 - 2500                | MPa        | $E$        |
| _Poisson's ratio_              | 0.16 - 0.35               | -          | $\nu$      |
| _Maximum swelling pressure_    | 3.2 - 13                  | MPa        | $\sigma$   |
| _Permeability_                 | 1 $\times$ 10$^{-14}$ - 1 $\times$ 10$^{-12}$ | m$^2$ | $k$        |
| _Initial water saturation_     | 0.1 - 0.5                 | -          | $S_{wi}$   |


# Output Directory  and Project File

In [71]:
# file's name
prj_name = "swelling_staufen.prj"

out_dir = os.environ.get("OGS_TESTRUNNER_OUT_DIR", "_out")
os.makedirs(out_dir, exist_ok=True)

In [72]:
# Find all .vtu files in the current directory
vtu_files = glob.glob("*.vtu")

# Copy each .vtu file to the output directory
for vtu_file in vtu_files:
    shutil.copy(vtu_file, out_dir)

print(f"Copied {len(vtu_files)} .vtu files to {out_dir}")

Copied 10 .vtu files to _out


In [73]:
ogs_path ='/Users/mollaali/dynamicPhasefield/build_release/bin'
ogs_util_path ='/Users/mollaali/dynamicPhasefield/build_release/bin'

# Post-processing

In [74]:
def calculate_principal_stresses(sigma_xx, sigma_yy, sigma_xy):
    # Calculate principal stresses
    lambda_1 = (sigma_xx + sigma_yy) / 2 + np.sqrt(((sigma_xx - sigma_yy) / 2) ** 2 + sigma_xy ** 2)
    lambda_2 = (sigma_xx + sigma_yy) / 2 - np.sqrt(((sigma_xx - sigma_yy) / 2) ** 2 + sigma_xy ** 2)
    lambda_3 = np.zeros_like(lambda_1)  # sigma_zz = 0
    
    # Calculate maximum principal stress
    sigma_max = np.max(np.abs([lambda_1, lambda_2, lambda_3]), axis=0)
    return sigma_max

def calculate_principal_strains(epsilon_xx, epsilon_yy, epsilon_xy):
    # Calculate principal strains
    lambda_1 = (epsilon_xx + epsilon_yy) / 2 + np.sqrt(((epsilon_xx - epsilon_yy) / 2) ** 2 + epsilon_xy ** 2)
    lambda_2 = (epsilon_xx + epsilon_yy) / 2 - np.sqrt(((epsilon_xx - epsilon_yy) / 2) ** 2 + epsilon_xy ** 2)
    lambda_3 = np.zeros_like(lambda_1)  # In 2D, assuming epsilon_zz = 0
    
    # Calculate maximum principal strain
    epsilon_max = np.max(np.abs([lambda_1, lambda_2, lambda_3]), axis=0)
    return epsilon_max

def calculate_displacement_magnitude(disp_x, disp_y):
    disp_magnitude = np.sqrt(disp_x ** 2 + disp_y ** 2)
    return disp_magnitude

In [75]:
def Post_processing(prefix, E, nu, K, swelling_stress_rate):
    reader = pv.get_reader(f"{out_dir}/{prefix}.pvd")

    porosity_data, saturation_data = [], []
    xs_data, ys_data, disp_x_data, disp_y_data, time_values = [], [], [], [], []
    sigma_xx_data, sigma_yy_data, sigma_zz_data, sigma_xy_data = [], [], [], []
    epsilon_xx_data, epsilon_yy_data, epsilon_zz_data, epsilon_xy_data = [], [], [], []

    for t, time_value in enumerate(reader.time_values):
        reader.set_active_time_value(time_value)

        mesh = reader.read()[0] 
        xs = mesh.points[:, 0]
        ys = mesh.points[:, 1]
        porosity = mesh.point_data["porosity"]
        saturation = mesh.point_data["saturation"]
        disp = mesh.point_data["displacement"]
        sigma = mesh.point_data["sigma"]  # Assuming "sigma" is the key for stress data
        epsilon = mesh.point_data["epsilon"]  # Assuming "epsilon" is the key for strain data

        disp_x = disp[:, 0]
        disp_y = disp[:, 1]
        sigma_xx = sigma[:, 0]
        sigma_yy = sigma[:, 1]
        sigma_zz = sigma[:, 2]
        sigma_xy = sigma[:, 3]
        epsilon_xx = epsilon[:, 0]
        epsilon_yy = epsilon[:, 1]
        epsilon_zz = epsilon[:, 2]
        epsilon_xy = epsilon[:, 3]

        xs_data.extend(xs)
        ys_data.extend(ys)
        porosity_data.extend(porosity)
        saturation_data.extend(saturation)
        disp_x_data.extend(disp_x)
        disp_y_data.extend(disp_y)
        sigma_xx_data.extend(sigma_xx)
        sigma_yy_data.extend(sigma_yy)
        sigma_zz_data.extend(sigma_zz)
        sigma_xy_data.extend(sigma_xy)
        epsilon_xx_data.extend(epsilon_xx)
        epsilon_yy_data.extend(epsilon_yy)
        epsilon_zz_data.extend(epsilon_zz)
        epsilon_xy_data.extend(epsilon_xy)
        time_values.extend([time_value] * len(xs))  
        
    # Calculate maximum principal strain for each point
    epsilon_max_data = calculate_principal_strains(np.array(epsilon_xx_data), np.array(epsilon_yy_data), np.array(epsilon_xy_data))

    # Calculate maximum principal stress for each point
    sigma_max_data = calculate_principal_stresses(np.array(sigma_xx_data), np.array(sigma_yy_data), np.array(sigma_xy_data))

    # Calculate displacement magnitude for each point
    disp_magnitude_data = calculate_displacement_magnitude(np.array(disp_x_data), np.array(disp_y_data))

    # Constant values for E, K, swelling_stress_rate for each point in the mesh
    E_values = [E] * len(xs_data)
    nu_values = [nu] * len(xs_data)
    K_values = [K] * len(xs_data)
    swelling_stress_rate_values = [swelling_stress_rate] * len(xs_data)


    #     data = list(zip(time_values, E_values, nu_values, K_values, swelling_pressure_values, porosity_avg, Saturation_avg, xs_data, ys_data,  disp_x_data, disp_y_data,
    #                    sigma_xx_data, sigma_yy_data, sigma_zz_data, sigma_xy_data,
    #                     epsilon_xx_data, epsilon_yy_data, epsilon_zz_data, epsilon_xy_data,
    #                     epsilon_max_data, sigma_max_data, disp_magnitude_data))

    data = list(zip(time_values,  E_values, nu_values, K_values, swelling_pressure_values, porosity_data, saturation_data, xs_data, ys_data, epsilon_max_data))

    # Write the data to a CSV file
    with open(f'{out_dir}/{prefix}.csv', 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
    #         csvwriter.writerow(['time_value', 'E', 'nu', 'K', 'swelling_pressure' , 'porosity_avg', 'Saturation_avg', 'xs', 'ys', 'disp_x', 'disp_y',
    #                             'sigma_xx', 'sigma_yy', 'sigma_zz', 'sigma_xy',
    #                             'epsilon_xx', 'epsilon_yy', 'epsilon_zz', 'epsilon_xy',
    #                             'principal_epsilon_max', 'principal_sigma_max', 'disp_magnitude']) 

        csvwriter.writerow(['time_value', 'E', 'nu', 'K', 'swelling_pressure' , 'porosity_avg', 'Saturation_avg',  'xs', 'ys', 'principal_epsilon_max', 'disp_magnitude'])  
        csvwriter.writerows(data)

In [82]:
class OGSModel:
    def __init__(self, prj_file):
        self.prj_file = prj_file
        self.tree = ET.parse(prj_file)
        self.root = self.tree.getroot()

    def _get_root(self):
        return self.root

    def _get_parameter_pointer(self, root, name, parameterpath):
        for parameter in root.findall(parameterpath):
            if parameter.find('name').text == name:
                return parameter
        return None

    def _set_type_value(self, parameterpointer, value, valuetag="values"):
        if parameterpointer is not None:
            value_element = parameterpointer.find(valuetag)
            if value_element is not None:
                value_element.text = ' '.join(map(str, value))
            else:
                ET.SubElement(parameterpointer, valuetag).text = ' '.join(map(str, value))

    def replace_parameter(self, name=None, value=None, valuetag="values"):
        root = self._get_root()
        parameterpath = ".//parameter"
        parameterpointer = self._get_parameter_pointer(root, name, parameterpath)
        self._set_type_value(parameterpointer, value, valuetag=valuetag)

    def replace_property_value(self, media_id, phase_type, property_name, values, valuetag="values"):
        root = self._get_root()
        for medium in root.findall(".//media/medium"):
            if medium.get('id') == media_id:
                for phase in medium.findall(".//phase"):
                    if phase.find('type').text == phase_type:
                        for property in phase.findall(".//property"):
                            if property.find('name').text == property_name:
                                target_element = property.find(valuetag)
                                if target_element is not None:
                                    target_element.text = ' '.join(map(str, values))
                                else:
                                    ET.SubElement(property, valuetag).text = ' '.join(map(str, values))

    def replace_property_value_general(self, xpath_query, value):
        root = self._get_root()
        target_element = root.find(xpath_query)
        if target_element is not None:
            target_element.text = str(value)

    def save(self, output_file):
        self.tree.write(output_file)


# Run the Simulation 


In [77]:
import pyvista as pv
pv.set_plot_theme("document")
if "PYVISTA_HEADLESS" in os.environ:
    pv.start_xvfb()
pv.set_jupyter_backend("static")

def swelling_numerical(E, nu, K, swelling_pressure, p_b, out_dir):

    #generate prefix from properties
    prefix = f"swelling_E{E[0]}_nu{nu[0]}_K{K[0]}_S{swelling_pressure[0]}"
    Screenoutput = f"log_swelling_E{E[0]}_nu{nu[0]}_K{K[0]}_S{swelling_pressure[0]}"
    print(prefix)
    print(Screenoutput)

    
    model = OGSModel(prj_name)
    # Replace the parameters in the model
    model.replace_parameter(name="E_l1", value=E)
    model.replace_parameter(name="nu1", value=nu)
    model.replace_parameter(name="permeability_l1", value=K) 
    model.replace_property_value(media_id="1", phase_type="Solid", property_name="swelling_stress_rate", values=swelling_pressure, valuetag="swelling_pressures")
    model.replace_property_value_general(".//media/medium[@id='1']//property[name='saturation']//p_b", p_b)

    model.save(f"{out_dir}/{prj_name}")
    print(f"Saved {prj_name}")
    
    model = ogs.OGS(INPUT_FILE=f"{out_dir}/{prj_name}", PROJECT_FILE=f"{out_dir}/{prj_name}", MKL=True, args=f"-o {out_dir}")
    model.replace_text(prefix, xpath="./time_loop/output/prefix")
    model.write_input() 

    #run simulation with ogs
    t0 = time.time()
    print(">>> OGS started execution ... <<<")
    !{ogs_path}/ogs {out_dir}/{prj_name} -o {out_dir} > {out_dir}/{Screenoutput}
    tf = time.time()
    print(">>> OGS terminated execution  <<< Elapsed time: ", round(tf - t0, 2), " s.")
    
    print("Post-processing")
    Post_processing(prefix, E, nu, K, swelling_pressure)

In [78]:
# Most probable values
most_probable_values = {
    'Young’s modulus (E)': 1000,
    'Poisson’s ratio (ν)': 0.2,
    'Maximum swelling pressure (σ)': 8,
    'Permeability (k)': 8e-13,
    'Air entry pressure (p_b)': 2000
}

# Ranges
ranges = {
    'Young’s modulus (E)': (500, 2500),
    'Poisson’s ratio (ν)': (0.16, 0.35),
    'Maximum swelling pressure (σ)': (3.2, 13),
    'Permeability (k)': (1e-14, 1e-12),
    'Air entry pressure (p_b)': (1000, 3500)
}

# Standard deviations as 1/4th of the range
std_devs = {param: (max_val - min_val) / 4 for param, (min_val, max_val) in ranges.items()}

def generate_values(mean, std_dev, min_val, max_val, num_values=20):
    values = np.random.normal(loc=mean, scale=std_dev, size=num_values-2)
    values = np.clip(values, min_val, max_val)
    values = np.concatenate(([min_val], values, [max_val]))
    values = np.unique(values)
    return np.sort(values)

# Generate 15 values for each parameter
generated_values = {}
for param, mean in most_probable_values.items():
    std_dev = std_devs[param]
    min_val, max_val = ranges[param]
    values = generate_values(mean, std_dev, min_val, max_val, num_values=15)
    generated_values[param] = values

# Convert values to Pa
swelling_pressure_values = (generated_values['Maximum swelling pressure (σ)'] * 1e6).tolist()
young_modulus_values = (generated_values['Young’s modulus (E)']* 1e6).tolist()
air_pressure_values = (generated_values['Air entry pressure (p_b)']).tolist()


# Create parameter sets from generated values
parameters = {
    "E": [[val, val, val] for val in young_modulus_values], 
    "nu": [[val, val, val] for val in generated_values["Poisson’s ratio (ν)"].tolist()], 
    "swelling_pressure": [[val, val, val] for val in swelling_pressure_values], 
    "K": [[val, val] for val in generated_values["Permeability (k)"].tolist()], 
    "p_b": [val for val in air_pressure_values], 

}

# Display the generated values for verification
for param, values in parameters.items():
    print(f"{param}:")
    if param == "p_b":
        for value in values:
            print(value)
    else:
        for value in values:
            print(" ".join(map(str, value)))
    print()


E:
500000000.0 500000000.0 500000000.0
599463258.6347227 599463258.6347227 599463258.6347227
623465981.0699878 623465981.0699878 623465981.0699878
672294819.2139126 672294819.2139126 672294819.2139126
905682034.8631377 905682034.8631377 905682034.8631377
946648341.0904171 946648341.0904171 946648341.0904171
1080551716.1829371 1080551716.1829371 1080551716.1829371
1236095596.5104084 1236095596.5104084 1236095596.5104084
1370228228.1385486 1370228228.1385486 1370228228.1385486
1384279209.9251482 1384279209.9251482 1384279209.9251482
1539491588.141038 1539491588.141038 1539491588.141038
1664063667.779906 1664063667.779906 1664063667.779906
1851900301.8454177 1851900301.8454177 1851900301.8454177
2500000000.0 2500000000.0 2500000000.0

nu:
0.16 0.16 0.16
0.21137413685893044 0.21137413685893044 0.21137413685893044
0.21164082864508876 0.21164082864508876 0.21164082864508876
0.21541344598924356 0.21541344598924356 0.21541344598924356
0.21565272934038163 0.21565272934038163 0.21565272934038163

In [81]:
print("=====================================")
print("E (MPa)\t\tν\t\tK (m²)\t\tSwelling Pressure (Pa)\t\tAir entry pressure (Pa)")
print("=====================================")
for E in parameters["E"]:
    for nu in parameters["nu"]:
        for swelling_pressure in parameters["swelling_pressure"]:
            for K in parameters["K"]:
                for p_b in parameters["p_b"]:
                    print(f"{E[0]:.2f}\t\t{nu[0]:.2f}\t\t{K[0]:.2e}\t\t{swelling_pressure[0]:.2e}\t\t{p_b:.2e}")
                    swelling_numerical(E, nu, K, swelling_pressure, p_b, out_dir)

E (MPa)		ν		K (m²)		Swelling Pressure (Pa)		Air entry pressure (Pa)
500000000.00		0.16		1.00e-14		3.20e+06		1.00e+03
swelling_E500000000.0_nu0.16_K1e-14_S3200000.0
log_swelling_E500000000.0_nu0.16_K1e-14_S3200000.0
Saved swelling_staufen.prj
>>> OGS started execution ... <<<
^C
>>> OGS terminated execution  <<< Elapsed time:  8.07  s.
Post-processing
500000000.00		0.16		1.00e-14		3.20e+06		1.33e+03
swelling_E500000000.0_nu0.16_K1e-14_S3200000.0
log_swelling_E500000000.0_nu0.16_K1e-14_S3200000.0
Saved swelling_staufen.prj
>>> OGS started execution ... <<<
^C
>>> OGS terminated execution  <<< Elapsed time:  1.08  s.
Post-processing
500000000.00		0.16		1.00e-14		3.20e+06		1.38e+03
swelling_E500000000.0_nu0.16_K1e-14_S3200000.0
log_swelling_E500000000.0_nu0.16_K1e-14_S3200000.0
Saved swelling_staufen.prj
>>> OGS started execution ... <<<
^C
>>> OGS terminated execution  <<< Elapsed time:  5.73  s.
Post-processing
500000000.00		0.16		1.00e-14		3.20e+06		1.41e+03
swelling_E500000000.0_nu0.16

UnboundLocalError: cannot access local variable 'child' where it is not associated with a value

# Combine  all valid CSV files into a single CSV file

In [ ]:
csv_files = glob.glob(f"{out_dir}/*.csv")
dataframes = []

for file in csv_files:
    try:
        data = pd.read_csv(file)
        if not data.empty:
            dataframes.append(data)
        else:
            print(f"File '{file}' is empty. Skipping...")
    except pd.errors.EmptyDataError:
        print(f"File '{file}' is empty or has formatting issues. Skipping...")

if dataframes:
    combined_data = pd.concat(dataframes, ignore_index=True)
    combined_data.to_csv("combined_data.csv", index=False)
    print("CSV files successfully combined and saved as 'combined_data.csv'")
else:
    print("No valid CSV files found or all files are empty.")